## Hello!

I'm AmirHossein Rezaaie with Student-Number 403541474 ; 

Project Titlle : Data Structures Project  

In [4]:
strlist = input().replace(" ", "").replace("\t", "").replace("\n", "")
print(f"Input String: {strlist}")


Input String: (2+3)*4^2+√16


In [5]:
normal_input = ""
i = 0
while i < len(strlist):
    current = strlist[i]

    if current in '0123456789()*/^√':
        normal_input += current
        i += 1

    elif current in '+-':
        prev_char = strlist[i-1] if i > 0 else ''
        if prev_char in '0123456789)':
            normal_input += current
            i += 1
        else:
            count_minus = 0
            while i < len(strlist) and strlist[i] in '+-':
                if strlist[i] == '-':
                    count_minus += 1
                i += 1
            if count_minus % 2 == 1:
                normal_input += '-'
            else:
                normal_input += '+'
                
    else:
        normal_input += current
        i += 1

print(f"Normal Input: {normal_input}")

Normal Input: (2+3)*4^2+√16


In [6]:
class Token:
    def __init__(self, type_, value):
        self.type = type_
        self.value = value

    def __repr__(self):
        return f"Token({self.type}, {repr(self.value)})"

    def __str__(self):
        return f"{self.type}: {self.value}"

tokens = []
i = 0
while i < len(normal_input):
    ch = normal_input[i]

    # عدد
    if ch in "0123456789":
        num = 0
        while i < len(normal_input) and normal_input[i] in "0123456789":
            num = num * 10 + int(normal_input[i])
            i += 1
        tokens.append(Token("NUMBER", num))
        continue

    # عملگرها
    elif ch in "+-*/^":
        tokens.append(Token("OPERATOR", ch))
        i += 1

    # رادیکال
    elif ch == "√":
        tokens.append(Token("OPERATOR", "√"))
        i += 1

    # پرانتزها
    elif ch == "(":
        tokens.append(Token("LPAREN", "("))
        i += 1
    elif ch == ")":
        tokens.append(Token("RPAREN", ")"))
        i += 1

    else:
        print(f"کاراکتر ناشناخته: {ch}")
        i += 1

print(f"Tokens : {tokens}")

Tokens : [Token(LPAREN, '('), Token(NUMBER, 2), Token(OPERATOR, '+'), Token(NUMBER, 3), Token(RPAREN, ')'), Token(OPERATOR, '*'), Token(NUMBER, 4), Token(OPERATOR, '^'), Token(NUMBER, 2), Token(OPERATOR, '+'), Token(OPERATOR, '√'), Token(NUMBER, 16)]


In [7]:
def precedence(op):
    if op in ("+", "-"): return 1
    if op in ("*", "/"): return 2
    if op == "^":        return 3
    if op == "√":        return 4
    return 0

stack = []
output_list = []

for token in tokens:
    if token.type == "LPAREN":
        stack.append(token.value)
        
    elif token.type == "RPAREN":
        while stack and stack[-1] != "(":
            output_list.append(stack.pop())
        if stack:
            stack.pop()
            
    elif token.type == "NUMBER":
        output_list.append(token.value)
        
    elif token.type == "OPERATOR":
        current_op = token.value
        while stack and stack[-1] != "(":
            top_op = stack[-1]
            if (precedence(top_op) > precedence(current_op) or
                (precedence(top_op) == precedence(current_op) and current_op not in ("^", "√"))):
                output_list.append(stack.pop())
            else:
                break
        stack.append(current_op)
        
while stack:
    output_list.append(stack.pop())

print(f"Output Lise is: {output_list}")

Output Lise is: [2, 3, '+', 4, 2, '^', '*', 16, '√', '+']


In [8]:
class Node:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None


def build_expression_tree(postfix):
    stack = []
    
    for token in postfix:
        if isinstance(token, int):
            stack.append(Node(token))
        else:
            node = Node(token)
            
            if token == "√":
                if not stack:
                    raise ValueError("Invalid expression: missing operand for √")
                node.left = stack.pop()
                node.right = None
            else:
                if len(stack) < 2:
                    raise ValueError("Invalid expression: not enough operands")
                right = stack.pop()
                left = stack.pop()
                node.left = left
                node.right = right
            
            stack.append(node)
    
    if len(stack) != 1:
        raise ValueError("Invalid expression: multiple roots remain")
    
    return stack[0]

In [9]:
root = build_expression_tree(output_list)

def print_tree(node, level=0, prefix="Root: "):
    if node:
        print("  " * level + prefix + str(node.value))
        print_tree(node.left, level + 1, "L-- ")
        print_tree(node.right, level + 1, "R-- ")

print_tree(root)

Root: +
  L-- *
    L-- +
      L-- 2
      R-- 3
    R-- ^
      L-- 4
      R-- 2
  R-- √
    L-- 16


In [10]:
import math

def evaluate_tree(node):
    if node is None:
        return 0
    
    if isinstance(node.value, int):
        return node.value

    if node.value == "+":
        return evaluate_tree(node.left) + evaluate_tree(node.right)
    if node.value == "-":
        return evaluate_tree(node.left) - evaluate_tree(node.right)
    if node.value == "*":
        return evaluate_tree(node.left) * evaluate_tree(node.right)
    if node.value == "/":
        right_val = evaluate_tree(node.right)
        if right_val == 0:
            raise ZeroDivisionError("Division by zero occurred!")
        return evaluate_tree(node.left) / right_val
    if node.value == "^":
        return math.pow(evaluate_tree(node.left), evaluate_tree(node.right))
    if node.value == "√":
        val = evaluate_tree(node.left)
        if val < 0:
            raise ValueError("Cannot take square root of negative number")
        return math.sqrt(val)
    
    raise ValueError(f"Unknown operator: {node.value}")

In [11]:
try:
    root = build_expression_tree(output_list)
    print("Expression tree built successfully!")
    
    result = evaluate_tree(root)
    print(f"Final result = {result}")

except ZeroDivisionError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"Syntax or calculation error: {e}")

Expression tree built successfully!
Final result = 84.0


In [12]:
from graphviz import Digraph

def visualize_tree(node, graph=None, parent=None, graph_id=0):
    if graph is None:
        graph = Digraph()
        graph.attr(rankdir='TB', size='10')
    
    current_id = str(id(node))
    label = str(node.value)
    graph.node(current_id, label)
    
    if parent is not None:
        graph.edge(parent, current_id)
    
    if node.left:
        visualize_tree(node.left, graph, current_id)
    if node.right:
        visualize_tree(node.right, graph, current_id)
    
    return graph

In [13]:
try:
    root = build_expression_tree(output_list)
    dot = visualize_tree(root)
    dot.render('expression_tree', format='png', cleanup=True)
    print("درخت با موفقیت ذخیره شد → فایل expression_tree.png")
except Exception as e:
    print(f"خطا در نمایش درخت: {e}")

درخت با موفقیت ذخیره شد → فایل expression_tree.png


In [14]:
variables = {}  # مثلاً: {"x": 10, "count": 5}

def add_variables(var_input):
    global variables
    variables = {}
    if not var_input.strip():
        return
    pairs = var_input.split(',')
    for pair in pairs:
        if '=' in pair:
            name, val = pair.split('=', 1)
            variables[name.strip()] = float(val.strip())

def evaluate_tree_with_vars(node):
    if node is None:
        return 0
    
    if isinstance(node.value, int):
        return node.value
    
    if node.value.isalpha() or '_' in node.value:
        return variables.get(node.value, 0)
    
    left_val = evaluate_tree_with_vars(node.left)
    if node.right:
        right_val = evaluate_tree_with_vars(node.right)
    else:
        right_val = None
    
    if node.value == "+": return left_val + right_val
    if node.value == "-": return left_val - right_val
    if node.value == "*": return left_val * right_val
    if node.value == "/":
        if right_val == 0:
            raise ZeroDivisionError("Division by zero!")
        return left_val / right_val
    if node.value == "^": return left_val ** right_val
    if node.value == "√": return math.sqrt(left_val)
    
    raise ValueError(f"Unknown operator or variable: {node.value}")